In [1]:
import numpy as np
import keras
from keras import layers
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input, Concatenate
from keras.models import Model

2024-02-04 01:51:00.115455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 01:51:00.393598: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 01:51:01.076542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-04 01:51:01.076603: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [26]:
def global_branch(input_shape):
  densenet121 = tf.keras.applications.DenseNet121(input_shape= input_shape, include_top= False, weights="imagenet")
  densenet121._name= 'densenet121_global_branch'
  return densenet121

In [27]:
def local_branch(input_shape):
  densenet121 = tf.keras.applications.DenseNet121(input_shape= input_shape, include_top= False, weights="imagenet")
  densenet121._name= 'densenet121_local_branch'
  return densenet121

In [7]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Permute, multiply
import tensorflow.keras.backend as K

def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x


In [9]:
img_shape= (224,224,3)
local_encoder = local_branch(img_shape)
global_encoder = global_branch(img_shape)

In [22]:
def model_fusion(local_encoder, global_encoder):
#   global_encoder._name = "Level_1"
#   local_encoder._name = "Level_2"

  for layer in local_encoder.layers:
      layer._name = layer.name + str("_2")

  local_features = local_encoder.output
  global_features = global_encoder.output
  concatenated_volume = Concatenate(axis=-1)([local_features, global_features])

  se_block_out = squeeze_excite_block(concatenated_volume)
  # Clasification fusion
  x_F = layers.Flatten()(se_block_out)
  classification_fusion =  layers.Dense(15, activation="sigmoid")(x_F)

  fusion_model = Model(
      inputs=[local_encoder.input, global_encoder.input], outputs= [classification_fusion]
  )

  return fusion_model

In [23]:
f_model = model_fusion(local_encoder, global_encoder)

In [24]:
f_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1_2_2 (InputLayer)       [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1_2_2 (Conv2D)      (None, 224, 224, 64  1792        ['input_1_2_2[0][0]']            
                                )                                                           